<h1>Автодополнение запросов</h1>

Работать буду по следующей схеме: сначала создаю алгоритм, который строит структуру-индекс, которую затем уже можно использовать в подсказках для поиска. Индекс на больших текстах может строится долго, но его последующее использование будет быстро, оно сводится к обращению по ключу в словаре.
Для начала применю следующий алгоритм: для каждого слова текста вычислю, какие другие слова чаще всего следуют за ним. Например, очевидно, что в большнстве случаев за словом "так" следует "же", за словом "потому" - "что", поэтому такая рекомендация имеет право на существование. Однако же она не учитывает предыдущий контекст, её работа зависит только от последнего введённого слова, это, явно, не соответствует тому, как строятся реальные высказывания (хотя поисковые запросы не очень длинны, в них эта модель может давать неплохие результаты). Модель можно расширить: Построить индекс по тому, какие слова чаще всего следуют за сочетаниями из двух, трех, четырех слов. Если мы обнаружили в тексте запроса какую-то последовательность из 4 слов, которая хранится у нас в индексе - скорее всего следующее введённое слово именно тем, которое мы храним в индексе, как наиболее часто встречающееся. Для построения таких 4-словных индексов нужны большие тексты, из которых можно достать сильные словосочетания, в которых можно почерпнуть надёжную статистику.
Дополнительно отмечу, что решал задачу автодополнения, как задачу подбора одного слова - следующие слова можно подобрать после того, как было подобрано оно

In [ ]:
data = open('Onegin.txt').readlines()
data[0:10]

['\n',
 'Александр Сергеевич Пушкин\n',
 'Евгений Онегин\n',
 'Роман в стихах\n',
 '\n',
 'Pe€tri de vanite€ il avait encore plus de cette espe`ce d’orgueil qui fait avouer avec la me^me indiffe€rence les bonnes comme les mauvaises actions, suite d’un sentiment de supe€riorite€, peut-e^tre imaginaire.\n',
 'Tire€ d’une lettre particulie`re[1]He мысля гордый свет забавить,\n',
 'Вниманье дружбы возлюбя,\n',
 'Хотел бы я тебе представить\n',
 'Залог достойнее тебя,\n']

In [ ]:
words_to_seq = {}

for line in range(len(data)):
  words = data[line].lower().split()
  if line < len(data) - 1:
    next_line = data[line + 1].lower().split()
    if len(next_line) != 0:
      words.append(next_line[0])
  for word in range(len(words) - 1):
    if not words[word].isalpha():
      if not (words[word][0:len(words[word]) - 1].isalpha() and words[word][-1] == ','):
        break
      words[word] = words[word][0:len(words[word]) - 1]
    if words[word] not in words_to_seq:
      words_to_seq[words[word]] = {}
    if words[word + 1] not in words_to_seq[words[word]]:
      words_to_seq[words[word]][words[word + 1]] = 0
    words_to_seq[words[word]][words[word + 1]] += 1
words_to_seq['так']

{'бедный': 1,
 'благочестия': 1,
 'бледно,': 1,
 'близко!..': 1,
 'бури': 1,
 'в': 1,
 'ваша': 1,
 'величавы,': 1,
 'видно,': 1,
 'возможно,': 1,
 'воспитаньем,': 1,
 'деревцо': 1,
 'доверчива': 1,
 'думал': 2,
 'если': 1,
 'же': 7,
 'за': 1,
 'зайчик': 1,
 'и': 6,
 'исправляется': 1,
 'как': 1,
 'ли': 1,
 'ли,': 1,
 'любит…': 1,
 'люди': 1,
 'медленно': 1,
 'меня': 1,
 'мило': 1,
 'мой': 1,
 'мысль': 1,
 'на': 1,
 'нас': 1,
 'наше': 1,
 'не': 1,
 'непорочны,': 1,
 'неприступны': 1,
 'никого': 1,
 'нимало': 1,
 'одевает': 1,
 'он': 2,
 'они': 1,
 'осмотрительны,': 1,
 'подшутил': 1,
 'полдень': 1,
 'привык': 1,
 'проповедовал': 1,
 'пряником': 1,
 'пчел': 1,
 'равнодушна,': 1,
 'смела?': 1,
 'смиренно': 1,
 'согласно…': 1,
 'тихо,': 1,
 'точно': 3,
 'точны,': 1,
 'ты,': 1,
 'уж': 1,
 'ужасно,': 1,
 'умны,': 1,
 'уносились': 1,
 'что': 1,
 'я,': 1}

In [ ]:
def get_continuation(initial_text):
  initial_words = initial_text.lower().split()
  last_word = initial_words[-1]
  raw_recs = words_to_seq[last_word]
  raw_recs[''] = 0
  top5_keys = ['', '', '', '', '']
  for rec in raw_recs:
    for current_top in range(len(top5_keys)):
      if raw_recs[top5_keys[current_top]] < raw_recs[rec]:
        top5_keys[current_top] = rec
        top5_keys = sorted(top5_keys, key=lambda x: raw_recs[x])
        break
  return top5_keys

In [ ]:
get_continuation('Сейчас он')

['пел', 'мне', 'был', 'не', 'в']

Неплохие рекомендации, каждое слово кажется довольно естественным продолжением.

In [ ]:
get_continuation('Весна')

['живит', 'в', 'и', 'весна!', 'моих']

Также попробовал алгоритм получения рекомендации, опирающийся на смысл слова. Для этого использовал модель word2vec русского языка, построенную по интернет-ресурсам. Применял её так: для каждого слова вычислял сумму векторов предшествующих ему в предложении слов так, чтобы наиболее "далёкие" (то есть расположенные в предложении дальше) слова имели меньший вес. Для слова получается набор векторов, построенных по предшествующим словам в разных предложениях. Усредняя эти вектора, получаю итоговый словарь, где ключи - слова текста, а значеия - 300-мерные представления, полученные на основании анализа предшествующих слов текста. Такой подход учитывает смысл слов, использует предыдущий контекст, а также в некоторой мере - частоту контекста (потому что в итоге вычисляется среднее значение по всем эмбеддингам).

In [ ]:
!pip install fasttext

     |████████████████████████████████| 68 kB 2.8 MB/s 
  Using cached pybind11-2.9.2-py2.py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3143551 sha256=81415171cb0ab7ffb8f60c530f289feb418a3fcc1637119e5816fea0cac6da40
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [ ]:
import fasttext
import fasttext.util

ft = fasttext.load_model('cc.ru.300.bin')

In [ ]:
import re

data = ' '.join(list(filter(lambda el: (len(el) > 0) and (not el.isspace()), open('Onegin.txt').readlines()))).lower()
data = re.split('[.!?\n]', data)
data = list(filter(lambda el: (len(el) > 0) and (not el.isspace()), data))
data[0:10]

['александр сергеевич пушкин',
 ' евгений онегин',
 ' роман в стихах',
 ' pe€tri de vanite€ il avait encore plus de cette espe`ce d’orgueil qui fait avouer avec la me^me indiffe€rence les bonnes comme les mauvaises actions, suite d’un sentiment de supe€riorite€, peut-e^tre imaginaire',
 ' tire€ d’une lettre particulie`re[1]he мысля гордый свет забавить,',
 ' вниманье дружбы возлюбя,',
 ' хотел бы я тебе представить',
 ' залог достойнее тебя,',
 ' достойнее души прекрасной,',
 ' святой исполненной мечты,']

In [ ]:
import numpy as np

emdedded_words = {}

for line in data:
  line = line.split()
  for word in range(1, len(line)):
    if line[word] not in emdedded_words:
      emdedded_words[line[word]] = {'vecs': np.array([0.0] * ft.get_dimension()), 'count': 0}
    prev_words_vecs = np.array([0.0] * ft.get_dimension())
    for prev_word in range(word):
      prev_words_vecs += ft.get_word_vector(line[prev_word]) * (2 ** (prev_word - word + 1))
    emdedded_words[line[word]]['vecs'] += prev_words_vecs
    emdedded_words[line[word]]['count'] += 1

In [ ]:
for emb in emdedded_words:
  emdedded_words[emb] = emdedded_words[emb]['vecs'] / emdedded_words[emb]['count']

In [ ]:
np.linalg.norm(emdedded_words['сон'] - emdedded_words['луна']), np.linalg.norm(emdedded_words['сон'] - emdedded_words['солнце'])

(0.8601517312619978, 1.9304960047973203)

In [ ]:
def get_k_nearest_neighbors(sentence, k):
  sentence = sentence.split()
  prev_words_vecs = np.array([0.0] * ft.get_dimension())
  for prev_word in range(len(sentence)):
    prev_words_vecs += ft.get_word_vector(sentence[prev_word]) * (2 ** (prev_word - len(sentence) + 1))
  return list(zip(*sorted(list(map(lambda key: (np.linalg.norm(prev_words_vecs - emdedded_words[key]), key), emdedded_words.keys())))))[0][:k]

print(get_k_nearest_neighbors('Ночь улица', 5))

('рядами', 'зимняя', 'стража', 'цветет', 'его:')


In [ ]:
get_k_nearest_neighbors('Сейчас он', 5)

('барщины', 'верил,', 'видит:', 'вслух,', 'говорил:')

Предсказания получились правдоподобными, глаголы очень подходят. В целом, учитывя небольшой размер текста и его стиль (алгоритмы явно адаптируются под особенности автора, что кажется правильным), остальные слова также подходят в качестве завершения фразы.

Теперь попытаюсь представить фразы исходного набора данных в виде векторов. Затем тем же способом буду строить вектор из запроса пользователя. По этому вектору буду искать наиболее близкие из уже имеющихся векторов.

In [3]:
import re

data = ' '.join(list(filter(lambda el: (len(el) > 0) and (not el.isspace()), open('Onegin.txt').readlines()))).lower()
expr = r'[^А-Яа-я .?!:,]'
parser = re.compile(expr)
data = re.sub(expr, '', data)
data = re.split('[.!?]', data)
data = list(filter(lambda el: (len(el) > 0) and (not el.isspace()), data))
data[0:10]

['александр сергеевич пушкин евгений онегин роман в стихах                        ,     ,  ',
 '     мысля гордый свет забавить, вниманье дружбы возлюбя, хотел бы я тебе представить залог достойнее тебя, достойнее души прекрасной, святой исполненной мечты, поэзии живой и ясной, высоких дум и простоты но так и быть рукой пристрастной прими собранье пестрых глав, полусмешных, полупечальных, простонародных, идеальных, небрежный плод моих забав, бессонниц, легких вдохновений, незрелых и увядших лет, ума холодных наблюдений и сердца горестных замет',
 ' глава первая и жить торопится, и чувствовать спешит',
 ' князь вяземский  мой дядя самых честных правил, когда не в шутку занемог, он уважать себя заставил и лучше выдумать не мог',
 ' его пример другим наука но, боже мой, какая скука с больным сидеть и день и ночь, не отходя ни шагу прочь',
 ' какое низкое коварство полуживого забавлять, ему подушки поправлять, печально подносить лекарство, вздыхать и думать про себя: когда же черт возьмет 

Видно, что предложения из романа довольно велики по сравнению с длиной типичного поискового запроса. Если их побить на более мелкие, получатся более похожие на запросы вектора.

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfvectorizer = TfidfVectorizer(analyzer='word')

In [5]:
expr = r'[^А-Яа-я ]'

all_sentences = []

parser = re.compile(expr)
for sentence in data:
  sentence = re.sub(expr, '', sentence)
  all_sentences.append(sentence)
all_sentences[0:10]

['александр сергеевич пушкин евгений онегин роман в стихах                               ',
 '     мысля гордый свет забавить вниманье дружбы возлюбя хотел бы я тебе представить залог достойнее тебя достойнее души прекрасной святой исполненной мечты поэзии живой и ясной высоких дум и простоты но так и быть рукой пристрастной прими собранье пестрых глав полусмешных полупечальных простонародных идеальных небрежный плод моих забав бессонниц легких вдохновений незрелых и увядших лет ума холодных наблюдений и сердца горестных замет',
 ' глава первая и жить торопится и чувствовать спешит',
 ' князь вяземский  мой дядя самых честных правил когда не в шутку занемог он уважать себя заставил и лучше выдумать не мог',
 ' его пример другим наука но боже мой какая скука с больным сидеть и день и ночь не отходя ни шагу прочь',
 ' какое низкое коварство полуживого забавлять ему подушки поправлять печально подносить лекарство вздыхать и думать про себя когда же черт возьмет тебя',
 '  так думал молодо

In [6]:
vectorizer = tfidfvectorizer.fit(all_sentences)

In [7]:
embedded_sentences = vectorizer.transform(all_sentences)
embedded_sentences

<1725x9316 sparse matrix of type '<class 'numpy.float64'>'
	with 21377 stored elements in Compressed Sparse Row format>

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

input_data = 'Бессмысленный и тусклый свет'

similarities = list(sorted(enumerate(cosine_similarity(vectorizer.transform([input_data]), embedded_sentences)[0]), key=lambda el: el[1]))

similar_sentences = []
for el in similarities[len(similarities) - 5:len(similarities)]:
  similar_sentences.append(all_sentences[el[0]])
similar_sentences

[' мой ваня моложе был меня мой свет а было мне тринадцать лет',
 ' и даже глупости смешной в тебе не встретишь свет пустой',
 ' свет решил что он умен и очень мил',
 ' довольно он морочил свет знаком он вам',
 ' что значит видеть свет']

Текст относительно невелик, поэтому предсказания не очень похожи на исходное предложение (хотя последнее созвучно по тематике). Исходный текст, например, не содержит слово "бессмысленный", то есть по факту идёт поиск по совпадению одного слова. Если построить вектора по огромному массиву текстов, можно получить результаты получше: будут попадаться похожие предложения, совпадения выведутся в результат.

In [13]:
def clean_words(words_list):
  answer = ['']
  for word in words_list:
    for char in word:
      if char.isalpha():
        answer[-1] += char
    answer.append('')
  return list(filter(lambda w: len(w) > 0, answer))

Теперь, чтобы сделать рекомендацию, из полученных похожих предложений вырежем лишнюю часть.

In [14]:
recs = []

for sentence in similar_sentences:
  sentence = clean_words(sentence.lower().split())
  word = 0
  while word in range(len(sentence)):
    if sentence[word] == input_data.split()[-1]:
      break
    word += 1
  recs.append(' '.join(sentence[word + 1:]))
list(filter(lambda el: len(el) > 0, recs))

['мой', 'ваня', 'моложе', 'был', 'меня', 'мой', 'свет', 'а', 'было', 'мне', 'тринадцать', 'лет']
['и', 'даже', 'глупости', 'смешной', 'в', 'тебе', 'не', 'встретишь', 'свет', 'пустой']
['свет', 'решил', 'что', 'он', 'умен', 'и', 'очень', 'мил']
['довольно', 'он', 'морочил', 'свет', 'знаком', 'он', 'вам']
['что', 'значит', 'видеть', 'свет']


['а было мне тринадцать лет',
 'пустой',
 'решил что он умен и очень мил',
 'знаком он вам']

In [17]:
input_data = 'и я любовь узнал душой с ее небесною отрадой с ее мучительной тоской'

similarities = list(sorted(enumerate(cosine_similarity(vectorizer.transform([input_data]), embedded_sentences)[0]), key=lambda el: el[1]))

similar_sentences = []
for el in similarities[len(similarities) - 5:len(similarities)]:
  similar_sentences.append(all_sentences[el[0]])
similar_sentences

[' не ты ль с отрадой и любовью слова надежды мне шепнул',
 ' кровь ее застыла',
 ' и где теперь ее сестра',
 ' я предан вам душой',
 ' ничто ее не занимает ее души не шевелит']

Текст того же автора дал очень правдоподобные результаты.

Пока что самые лучшие, на мой взгляд, результаты дал первый алгоритм - анализ частоты втречаемости комбинаций слов. Разовью идею, увеличивая количество слов, которые определяют рекомендуемое следующее. Построю 4 словаря-индекса, в них будут лежать сочетания от 1 до 4 слов (смысла брать больше для поискового запроса, полагаю, нет). Рекомендация будет идти сверху вниз по количеству слов в сочетании: сначала будет произведён поиск в словаре из 4-словных комбинаций последовательных слова исходного текста. Если что-то найдётся, скорее всего пользователь вводит именно эту фразу. Если в словаре ничего не нашлось, спускаемся ниже, в словарь с длинами комбинаций в 3 слова. И так далее до индекса, в котором хрантся пары последовательно идущих слов.

In [19]:
data = ' '.join(open('Пугачев.txt').readlines())

In [20]:
alphabet = 'абвгдеёжзийклмнопртсухфцчшщьыьэюяabcdefghijklmnopqrstuvwxyz'
stop_signs = '.!?;:'
other_signs = ','

def looks_like_word(text):
  ok = False
  for letter in alphabet:
    if letter in text.lower():
      ok = True
      break
  return ok

def stop_separator(text):
  sep = ''
  for stop in stop_signs:
    if stop in text.lower():
      sep = stop
      break
  return sep

sentences = ['']
for token in data.split():
  if looks_like_word(token):
    sign_inside = stop_separator(token)
    if len(sign_inside) == 0:
      sentences[-1] += ''.join(token.split(other_signs)) + ' '
    else:
      mas = token.split(sign_inside)
      for elem in range(len(mas)):
        sentences[-1] += ''.join(mas[elem].split(other_signs)) + ' '
        if elem + 1 < len(mas):
          sentences.append('')
  else:
    if len(stop_separator(token)) != 0:
      sentences.append('')

sentences[10:60:5]

[' коего все затеи не от разума и воинского распорядка но от дерзости случая и удачи зависели ',
 ' Царская грамота ',
 ' Внутренние беспокойства ',
 ' течет к югу вдоль их цепи до того места где некогда положено было основание Оренбургу и где теперь находится Орская крепость ',
 ' Его течение быстро ',
 ' Они зимовали на ее берегах в то время еще покрытых лесом и безопасных по своему уединению ',
 ' казаки стали получать жен из татарских улусов ',
 ' Живя набегами окруженные неприязненными племенами казаки чувствовали необходимость в сильном покровительстве и в царствование Михаила Федоровича послали от себя в Москву просить государя чтоб он принял их под свою высокую руку ',
 ' Шах жаловался царю ',
 ' а на Яик посланы были стрельцы в последствии времени составившие с казаками одно племя ']

In [21]:
clear_sentences = []

for sentence in sentences:
  clear_sentences.append('')
  for symbol in sentence:
    if symbol.isalpha() or symbol == ' ':
      clear_sentences[-1] += symbol.lower()

clear_sentences[10:60:5]

[' коего все затеи не от разума и воинского распорядка но от дерзости случая и удачи зависели ',
 ' царская грамота ',
 ' внутренние беспокойства ',
 ' течет к югу вдоль их цепи до того места где некогда положено было основание оренбургу и где теперь находится орская крепость ',
 ' его течение быстро ',
 ' они зимовали на ее берегах в то время еще покрытых лесом и безопасных по своему уединению ',
 ' казаки стали получать жен из татарских улусов ',
 ' живя набегами окруженные неприязненными племенами казаки чувствовали необходимость в сильном покровительстве и в царствование михаила федоровича послали от себя в москву просить государя чтоб он принял их под свою высокую руку ',
 ' шах жаловался царю ',
 ' а на яик посланы были стрельцы в последствии времени составившие с казаками одно племя ']

In [22]:
two_words_combinations = {}

for sentence in clear_sentences:
  sentence = sentence.split()
  for word in range(2, len(list(filter(lambda w: len(w) > 0, sentence)))):
    two_words = sentence[word - 2] + sentence[word - 1]
    if two_words not in two_words_combinations:
      two_words_combinations[two_words] = {}
    if sentence[word] not in two_words_combinations[two_words]:
      two_words_combinations[two_words][sentence[word]] = 0
    two_words_combinations[two_words][sentence[word]] += 1

two_words_combinations['внем']

{'атаману': 1,
 'было': 2,
 'веселия': 1,
 'и': 1,
 'находилось': 1,
 'находится': 1,
 'начальник': 1,
 'робость': 1,
 'собрано': 1,
 'убежища': 1,
 'уже': 1}

Появляются уже более сложные конструкции: "пускались в море", "в разных местах", "в состоянии были".

In [23]:
def n_words_combinations(n):
  words_combinations = {}

  for sentence in clear_sentences:
    sentence = sentence.split()
    for word in range(n, len(list(filter(lambda w: len(w) > 0, sentence)))):
      prev_words = ''
      for delta in range(1, n + 1):
        prev_words += sentence[word - (n + 1 - delta)]
      if prev_words not in words_combinations:
        words_combinations[prev_words] = {}
      if sentence[word] not in words_combinations[prev_words]:
        words_combinations[prev_words][sentence[word]] = 0
      words_combinations[prev_words][sentence[word]] += 1
  
  return words_combinations
n_words_combinations(4)['россииохраняяюжныеее']

{'границы': 1}

Преимущество такого анализа в том, что не нужно производить сравнения векторов, высчитывать расстояния между ними, сортировать, то есть тратить время. Процесс построения происходит до, того, как пользователь отправит запрос.

Такой подход к тому же даёт возможность подсказывать пользователю, где совершены опечатки в запросе (если идти с начала запроса и сравнивать слова пользователя с наиболее часто встречающимися словами в данной комбинации через редакторское расстояние, то слабо отличающиеся слова можно заменсять на те, что есть в словаре, скорее всего пользователь в них ошибся).

Дополнительно, думаю, что лучше дообучивать любой из приведённых алгоритмов на наборах, которые генерирует сам пользователь. Манера разгоровора для каждого уникальна, нужно уметь подстраиваться под речь конкретного пользователя, чтобы программа вела себя естественно.

Также испытал другой подход: высчитывал не частоту употребления слов друг с другом, а средние расстояния между ними, вычисленные, как разности индексов.

In [39]:
words_distances = {}
for sentence in clear_sentences:
  sentence = list(filter(lambda w: len(w) > 0, sentence.split()))
  for i in range(len(sentence)):
    for j in range(i + 1, len(sentence)):
      word1, word2 = sentence[i], sentence[j]
      if word1 not in words_distances:
        words_distances[word1] = {}
      if word2 not in words_distances:
        words_distances[word2] = {}
      if word2 not in words_distances[word1]:
        words_distances[word1][word2] = [None, 0]
      if word1 not in words_distances[word2]:
        words_distances[word2][word1] = [None, 0]
      words_distances[word1][word2][0] = abs(i - j)
      words_distances[word1][word2][1] += 1
      words_distances[word2][word1][0] = abs(i - j)
      words_distances[word2][word1][1] += 1

for word1 in words_distances:
  for word2 in words_distances[word1]:
    words_distances[word1][word2] = words_distances[word1][word2][0] / words_distances[word1][word2][1]

closest = {}
for word1 in words_distances:
  top5_closest = list(sorted(words_distances[word1].keys(),
                             key=lambda word2: words_distances[word1][word2]))[0:5]
  closest[word1] = top5_closest

In [40]:
words_distances['благодеяния']

{'воздадим': 14.0,
 'же': 15.0,
 'за': 2.5,
 'к': 7.0,
 'любовь': 8.0,
 'матернюю': 9.0,
 'мы': 13.0,
 'нам': 0.5,
 'несказанные': 2.0,
 'сии': 4.0,
 'твои': 3.0,
 'твою': 10.0,
 'тебе': 12.0,
 'чем': 16.0}

In [41]:
closest['благодеяния']

['нам', 'несказанные', 'за', 'твои', 'сии']

Предсказания довольно хороши.

Помимо этого, можно применить ML вещи: сети, классические алгоритмы.